### Implementação : Preditor de Artigos de Palavras em Alemão

O modelo proposto tem como objetivo predizer o artigo de substantivos em alemão (Der - Masculino, Die - Feminino, Das - Neutro), cuja tarefa é bastante desafiadora para estudantes do idioma alemão pela falta de correlação dos gêneros das palavras em relação ao idioma português.

O objetivo desta implementação possui dois direcionamentos:

- Confirmação de padrões observados no idioma em relação à distribuição dos caracteres em uma palavra para determinado gênero.
- Determinação do gênero mais dificil de predizer por conta da presença de exceções presentes no idioma
- Discussão sobre memorização e dificuldade de memorização dos gêneros das palavras algoritmo x humano

Além da predição propriamente dita, a diferença de acurácia gerada neste modelo nos proverá insights valiosos sobre a presença de exceções às regras em relação às terminações das palavras em alemão

#### 1 - Importação das bibliotecas

Importaremos as bibliotecas necessárias para o projeto. Esta inclui numpy, pandas, pickle, scikit-learn e keras (tensorflow)

In [ ]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.layers import Input, Embedding, Dense, Flatten
from keras.layers import LSTM, Dropout
from keras.models import Model
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import accuracy_score


#### 2 - Carregamento dos Dados

Os dados estão serializados em formato json, e constituem de 2610 palavras alemãs com seus respectivos gêneros, balanceados de forma que contenha quantidades iguais para cada um
Para a extração, carregamos o arquivo serializado.

In [ ]:
def load_pickle(filename):
    infile = open(filename,'rb')
    objeto = pickle.load(infile)
    infile.close()
    return objeto

base = load_pickle('Worter.p')
base

#### 3 - Pré-Processamento dos Dados

##### 3.1 - Tratamento dos Caracteres

Devemos extrair somente as palavras e seus artigos, e armazena-los em um DataFrame Pandas

In [ ]:
#Extract words and articles
artikels = []
worter = []

for key, wort in base.items():
    artikels.append(wort['Gender'])
    worter.append(wort['ORTH'])
    
df = pd.DataFrame({'artikel': artikels,'wort': worter})
df

Após armazenamento, devemos garantir que todas as palavras possuam somente letras minúsculas

In [ ]:
#Cleaning Dataset
df = df.loc[(df.loc[:,'artikel'] == 'Der') | 
            (df.loc[:,'artikel'] == 'Die') |
            (df.loc[:,'artikel'] == 'Das'), :] 

df['wort'] = df['wort'].str.lower()
df

##### 3.2 - Treinamento e Teste

Para evitar a contaminação dos dados, separamos dois conjuntos de dados, sendo treinamento e teste

In [ ]:
#Split train and test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

train_texts = train_df['wort'].values 
test_texts = test_df['wort'].values 

##### 3.3 Tokenizador

Criamos um Tokenizer do Keras para trabalhar em nível de caracteres, onde cada Token equivale a um caractere. Treinando com os caracteres presentes nas palavras de treinamento, obtemos um dicionário de caracteres, incluindo as variantes alemãs ä, ö, ü e ß, e UNK para caracteres desconhecidos (oov_token como parâmetro)

In [ ]:
# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)
print(tk.word_index)

Checamos o tamanho do dicionário gerado, incluindo as 26 letras do alfabeto, 4 variantes alemãs e 1 UNK

In [ ]:
vocab_size = len(tk.word_index)
vocab_size

##### 3.4 Texto para Sequências

Os textos de treinamento e teste são convertidos em sequências de números inteiros utilizando os tokens gerados acima.

In [ ]:
# Convert string to index 
train_sequences = tk.texts_to_sequences(train_texts)
test_texts = tk.texts_to_sequences(test_texts)
test_texts[0:5]

 ##### 3.5 Padding
 
 As sequências de números inteiros dos textos de treinamento e teste são padronizadas para terem o mesmo comprimento máximo, definido como 25. Tanto as sequências de treinamento quanto as de teste são ajustadas para terem um comprimento máximo de 25, preenchendo os valores com zeros no início (padding='pre', garantindo que todas as sequências tenham o mesmo comprimento e que contemplem palavras grandes.

In [ ]:
# Padding
maxlen = 25

train_data = pad_sequences(train_sequences, maxlen=maxlen, padding='pre')
test_data = pad_sequences(test_texts, maxlen=maxlen, padding='pre')
test_data

##### 3.6 - Conversão para matriz Float

As sequências de treinamento e teste são convertidas em matrizes numpy do tipo float, a fim de compartibilizar com o modelo LSTM

In [ ]:
# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')
test_data

##### 3.7 - Tratamento das Classes

As classes dos conjuntos de treinamento e teste são então definidos como inteiros através do LabelEncoder

In [ ]:
train_classes = train_df['artikel'].values
test_classes = test_df['artikel'].values

le = LabelEncoder()
le = le.fit(df['artikel'])

train_classes = le.transform(train_classes)
test_classes = le.transform(test_classes)
test_classes[0:20]

As classes são então canonizadas através de OneHotEncoder

In [ ]:
train_classes = to_categorical(train_classes)
test_classes = to_categorical(test_classes)
test_classes

##### 3.8 - Atribuição dos Pesos

Construímos assim os pesos da camada de embedding para a rede neural. Em seguida, iteramos sobre cada palavra e índice no índice de palavras gerado pelo Tokenizer. Para cada palavra, é criado um vetor one-hot de zeros, onde a posição correspondente ao índice da palavra é definida como 1, para que seja convertida em uma matriz numpy, tornando-se os pesos da camada de embedding da rede neural. Esses pesos serão usados ​​para inicializar a camada de embedding da rede neural durante o treinamento.

In [ ]:
#Setar onehot para cada letra
embedding_weights = []
embedding_weights.append(np.zeros(vocab_size))
for char, i in tk.word_index.items():
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)
embedding_weights[0:5]

#### 4 - Treinamento do Modelo

Criamos a função responsável por criar e compilar o modelo da rede neural. Ela recebe o tamanho do vocabulário e o comprimento máximo das sequências. O modelo consiste em uma camada de embedding inicializada com os pesos predefinidos (embedding_weights), seguida por uma camada LSTM com ativação ReLU, uma camada Flatten, uma camada Dense com ativação ReLU e dropout, e finalmente uma camada Dense de saída com ativação softmax para a classificação em 3 classes. O modelo é compilado com a função de perda categorical_crossentropy e o otimizador adam

In [ ]:
# parameter 

embedding_size = 31
num_of_classes = 3

def save_pickle(filename, objeto):
    outfile = open(filename,'wb')
    pickle.dump(objeto,outfile)
    outfile.close()

def create_model(vocabulary_size, seq_len):  
    model = Sequential()
    model.add(Input(shape=(maxlen,)))
    model.add(Embedding(vocabulary_size, seq_len, weights=[embedding_weights], input_length=maxlen))
    model.add(LSTM(64, return_sequences=True, activation='relu'))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_of_classes,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])    
    return model

model = create_model(vocab_size+1, embedding_size)
model.summary()

São definidos dois callbacks para monitorar e ajustar o treinamento da rede neural: EarlyStopping e ReduceLROnPlateau

O primeiro monitora onloss durante o treinamento. Se a perda não diminuir após 10 épocas, o treinamento será interrompido e os pesos da melhor época serão restaurados (restore_best_weights=True).

O segundo callback, também monitora a perda. No entanto, se a perda não diminuir após 3 épocas, a taxa de aprendizado será reduzida em um fator de 0.1 (factor=0.1).

In [ ]:
es = EarlyStopping(monitor= 'loss', patience = 10, verbose = 1, restore_best_weights=True)
rlr = ReduceLROnPlateau(monitor='loss', factor= 0.1, patience= 3, verbose=1)

Por fim, o modelo é treinado com tamanho de lote 32 e por 50 épocas.

In [ ]:
#Treinamento do modelo
history = model.fit(train_data, train_classes,
                    validation_data=(test_data, test_classes),
                    batch_size=32,
                    epochs=50,
                    verbose=1,
                    callbacks=[es, rlr])


##### 5 - Avaliação dos dados de Teste

O modelo treinado é avaliado utilizando os dados de teste, obtendo assim a sua perda e acurácia

In [ ]:
accuracy = model.evaluate(test_data,test_classes)
accuracy

Geramos as predições para cada exemplo nos dados de teste

Em seguida, as previsões são convertidas em rótulos de classe usando np.argmax() para encontrar o índice da classe com a maior probabilidade para cada exemplo.

Depois, os rótulos de classe são invertidos para seus valores originais para revertar a canonização realizada durante o treinamento

In [ ]:
predictions = model.predict(test_data)
predictions = [np.argmax(x) for x in predictions]
predictions = le.inverse_transform(predictions)
predictions[0:5]

Assim, geramos o DataFrame com os artigos reais e os preditos pelo modelo

In [ ]:
results = pd.DataFrame({'Wort': test_df['wort'].values, 'Real Artikel': test_df['artikel'].values, 'Predicted Artikel': predictions})
results

#### 6 - Particionamento da Solução e Discussão dos Resultados

Obtemos as acurácias para cada um dos artigos Der, Die, Das

No idioma alemão, determinados gêneros possuem menos exceções às regras do que outros.

Sendo assim, espera-se que os resultados sejam na seguinte ordem de acurácia: Die > Der > Das

In [ ]:
results_der = results.loc[results['Real Artikel'] == 'Der']
accuracy_der = accuracy_score(results_der['Real Artikel'], results_der['Predicted Artikel'])
accuracy_der


In [ ]:
results_die = results.loc[results['Real Artikel'] == 'Die']
accuracy_die = accuracy_score(results_die['Real Artikel'], results_die['Predicted Artikel'])
accuracy_die

In [ ]:
results_das = results.loc[results['Real Artikel'] == 'Das']
accuracy_das = accuracy_score(results_das['Real Artikel'], results_das['Predicted Artikel'])
accuracy_das

Os resultados gerais de acurácia parecem um pouco desmotivadores, principalmente para o artigo neutro Das, que apresentou mais erros do que acertos.

Isso é esperado, uma vez que o idioma alemão apresenta diversas exceções às regras

No entanto, podemos observar alguns padrões de comportamento das palavras de acordo com o caractere e suas posições.

Filtramos assim as palavras com determinadas terminações, e calculamos a sua acurácia separadamente

In [ ]:
def custom_ending(wort_ending):
    result_textpart = results.loc[results['Wort'].str.endswith(wort_ending)]
    accuracy_textpart = accuracy_score(result_textpart['Real Artikel'], result_textpart['Predicted Artikel'])
    print(f'Accuracy: "{wort_ending}": {round(accuracy_textpart,2)}')

custom_ending('er') #Der
custom_ending('en') #Der
custom_ending('keit') #Die
custom_ending('heit') #Die
custom_ending('tät') #Die
custom_ending('e') #Die
custom_ending('chen') #Das
custom_ending('a') #Das

Os resultados de acurácia foram esperados, pois:
- Palavras com terminações er e en costumam ser masculinas, com muitas exceções às regras, tendendo a diminuir o acerto pelo modelo
- Todas as palavras terminadas com "keit", "heit" e "tät" são predominantemente femininas, sem exceções, tendendo ao 100% de acerto. Terminações com "e" apresenta algumas exceçoes.
- Palavras terminadas com "chen" e "a" costumam ser palavras de gênero neutro, com poucas exceções. 

#### 7 - Conclusão

Estudantes do idioma alemão se deparam com palavras que não seguem a mesma lógica de atribuição de gênero do português, buscando as vezes por padrões nas palavras que fornecem pistas para esta tarefa

Porém, o esforço atribuído nesta memorização pode não ser compensada, uma vez que este padrão pode apresentar muitas exceções às regras, como os artigos "er" e "en", enquanto que outro padrões podem possuir comportamento mais bem definido, como heit e keit. Este comportamento foi refletido no modelo treinado.

Assim, o modelo auxilia o aluno à a desenvolver uma compreensão mais profunda das nuances da língua alemã e a reconhecer quando as regras podem não se aplicar de forma estrita.